In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import numpy as np
import pandas as pd

import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from scipy.stats import norm

from utils import c2st_nn_fit, MatConvert, c2st_nn_power
from models import myLDA

### Blob Dataset 

In [8]:
sigma_mx2_standard = np.array([[0.03, 0], [0, 0.03]])
sigma_mx2 = np.zeros((9, 2, 2))
for i in range(9):
    sigma_mx2[i, :, :] = sigma_mx2_standard
    if i < 4:
        sigma_mx2[i, 0, 1] = -0.02 - 0.002*i
        sigma_mx2[i, 1, 0] = -0.02 - 0.002*i
    elif i > 4:
        sigma_mx2[i, 0, 1] = -0.02 + 0.002*(i-5)
        sigma_mx2[i, 1, 0] = -0.02 + 0.002*(i-5)
        
sigma_mx2[4, :, :] = sigma_mx2_standard

In [9]:
def sample_blobs_Q(N0, N1, sigma_mx_2, rows=3, cols=3, rs=None):
    """Generate Blob-D for testing type-II error (or test power)."""
    rs = check_random_state(rs)
    mu = np.zeros(2)
    sigma = np.eye(2) * 0.03
    X = rs.multivariate_normal(mu, sigma, size=N0)
    Y = rs.multivariate_normal(mu, np.eye(2), size=N1)
    # assign to blobs
    X[:, 0] += rs.randint(rows, size=N0)
    X[:, 1] += rs.randint(cols, size=N0)
    Y_row = rs.randint(rows, size=N1)
    Y_col = rs.randint(cols, size=N1)
    locs = [[0,0],[0,1],[0,2],[1,0],[1,1],[1,2],[2,0],[2,1],[2,2]]
    for i in range(9):
        corr_sigma = sigma_mx_2[i]
        L = np.linalg.cholesky(corr_sigma)
        ind = np.expand_dims((Y_row == locs[i][0]) & (Y_col == locs[i][1]), 1)
        ind2 = np.concatenate((ind, ind), 1)
        Y = np.where(ind2, np.matmul(Y,L) + locs[i], Y)
    return X, Y

In [10]:
# Setup seeds 
np.random.seed(1203)
torch.manual_seed(1203)
torch.cuda.manual_seed(1203)

torch.backends.cudnn.deterministic = True
is_cuda = True

# Setup for all experiments 
dtype = torch.float
device = torch.device("cuda:0")
alpha = 0.05 # test threshold 
ir_list = [1,3,5,7,9] # imbalance ratio
x_in = 2 # number of neurons in the input layer (dimension of data)
H = 50 # number of neurons in the hidden layer
x_out = 2 # number of neurons in the output layer 
K = 1 # number of experiments
n1 = 30 # size of minority samples

In [25]:
# For each n in n_list, train deep kernel and run two-sample test
stats_a_dict = {}
stats_d_dict = {}
stats_p_dict = {}

acc_opt_cuts = []
pwr_opt_cuts = []

a_powers = {}
d_powers = {}
p_powers = {}

for r in ir_list:
    np.random.seed(1203)
    N1 = 9 * n1 * r
    N2 = 9 * n1
    batch_size = N1 + N2 # not using batch 
    n_epoch_c2st = 4000
    
    # Repeat experiments K times (K = 10) and report average test powers
    stats_p_nn = []
    
    for kk in range(3):
        print('='*100)
        print("\t\t\t\tImbalance Ratio: {}".format(int(N1/N2)))
        print(f"\t\t\t\tExperiments{kk+1}")
        print('='*100)
        # Generate Blob-D
        np.random.seed(seed=112*kk + 1 + N1)
        s1, s2 = sample_blobs_Q(N1, N2, sigma_mx2)
        
        S = np.concatenate((s1,s2), axis=0)
        S = MatConvert(S, device, dtype)
        
        # Train C2ST-Power
        np.random.seed(seed=1203)
        torch.manual_seed(1203)
        torch.cuda.manual_seed(1203)
        y = (torch.cat((torch.zeros(N1, 1), torch.ones(N2, 1)), 0)).squeeze(1).to(device, dtype).long()
        
        pred, tau0, tau1, stat_c2st = c2st_nn_power(S, y, x_in, H, x_out, 0.005, n_epoch_c2st, batch_size, device, dtype)
        print("-"*100)
        print('-'*100)
        print("\t\t\t\t   Testing")
        print('-'*100)
        print('-'*100)
        
        # statistics
        stats_p_nn.append(stat_c2st)

    stats_p_dict[r] = stats_p_nn
    # power
    pwr = np.mean((stats_p_nn > (1-norm.ppf(alpha))))
    p_powers[r] = pwr
    print("Asymptotic power: {}".format(pwr))
    print("="*50)

				Imbalance Ratio: 1
				Experiments1
Epoch 100, tau0: 0.3556, tau1: 0.5333, Asymp Power Loss: 1.8831, Power Stats Training: 1.8672
Epoch 200, tau0: 0.3630, tau1: 0.4963, Asymp Power Loss: 2.2392, Power Stats Training: 2.3573
Epoch 300, tau0: 0.4815, tau1: 0.4296, Asymp Power Loss: 1.3587, Power Stats Training: 1.4684
Epoch 400, tau0: 0.4296, tau1: 0.4889, Asymp Power Loss: 1.3532, Power Stats Training: 1.3457
Epoch 500, tau0: 0.5185, tau1: 0.3704, Asymp Power Loss: 1.8073, Power Stats Training: 1.8579
Epoch 600, tau0: 0.4667, tau1: 0.4370, Asymp Power Loss: 1.6279, Power Stats Training: 1.5904
Epoch 700, tau0: 0.4593, tau1: 0.4444, Asymp Power Loss: 1.5772, Power Stats Training: 1.5899
Epoch 800, tau0: 0.5259, tau1: 0.3704, Asymp Power Loss: 1.7343, Power Stats Training: 1.7346
Epoch 900, tau0: 0.5259, tau1: 0.3704, Asymp Power Loss: 1.7344, Power Stats Training: 1.7346
Epoch 1000, tau0: 0.5259, tau1: 0.3704, Asymp Power Loss: 1.7345, Power Stats Training: 1.7346
Epoch 1100, tau0: 0

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the neural network architecture
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(2, 4)
        self.layer2 = nn.Linear(4, 8)
        self.layer3 = nn.Linear(8, 16)
        self.relu = nn.ReLU()
        self.fc_layer = nn.Linear(16, 2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.layer3(x)
        x = self.fc_layer(x)
        return x

# Define the training loop
def train(model, optimizer, criterion, train_loader, epochs):
    for epoch in range(epochs):
        running_loss = 0.0
        for data, target in train_loader:
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print('Epoch [%d], loss: %.3f' % (epoch + 1, running_loss))

In [ ]:
# Prepare the data
X_train, y_train = train_data.iloc[:, :-1].values, train_data.iloc[:, -1].values
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)

In [ ]:
# Initialize the model, optimizer, and loss function
model = NeuralNetwork()
optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
# Train the model
train(model, optimizer, criterion, train_loader, epochs=500)

In [ ]:
X_te = torch.tensor(X_te.values, dtype = torch.float32)

In [ ]:
model.eval()

In [ ]:
with torch.no_grad():
    pred = model(X_te)

In [ ]:
pred

In [ ]:
# n0 >> n1 
val_n0 = np.sum(y_te == 0) # majority sample
val_n1 = np.sum(y_te == 1) # minority sample 

In [ ]:
# Train neural network classifier for optimizing accuracy vs power
val_nn = MLPClassifier(hidden_layer_sizes=(3,2), max_iter=10000, verbose=True, early_stopping = True, validation_fraction=0.2, learning_rate='adaptive')
val_nn.fit(X_tr, y_tr)

In [ ]:
val_probs = val_nn.predict_proba(X_te)[:, 1]

In [ ]:
acc = np.zeros(len(cutvals))
power = np.zeros(len(cutvals))
for j in range(len(cutvals)):
    k = cutvals[j]
    tmp = (probs > k).astype(int)
    acc[j] = np.sum(tmp == y_te) / len(y_te)
    tau0 = np.mean(tmp[y_te == 0] == 1) # P_{X ~ q}(h(X) = 1 | Y = 0)
    tau1 = np.mean(tmp[y_te == 1] == 0) # P_{X ~ p}(h(X) = 0 | Y = 1)
    if int(tau1) == 1:
        power[j] = 0
    else:
        power[j] = (1 - tau0 - tau1) / np.sqrt(tau0 * (1 - tau0) / n0 + tau1 * (1 - tau1) / n1)


In [ ]:
# Optimal cutoff
acc_cut_opt = cutvals[np.argmax(acc)]
power_cut_opt = cutvals[np.argmax(power)]


In [ ]:
tmp

In [ ]:
# test 
test_n0 = np.sum(y_test == 0) # majority
test_n1 = np.sum(y_test == 1) # minority 

test_nn = MLPClassifier(hidden_layer_sizes=(3,2), max_iter=10000, verbose=True, early_stopping = True, validation_fraction=0.2, learning_rate='adaptive')
test_nn.fit(X_train, y_train)
test_probs = test_nn.predict_proba(X_test)[:, 1]

In [ ]:
# prediction
y_acc_pred = (test_probs > acc_cut_opt).astype(int)
y_power_pred = (test_probs > power_cut_opt).astype(int)
y_default_pred = (test_probs > 0.5).astype(int)


In [ ]:
from collections import Counter

Counter(y_acc_pred)


In [ ]:
Counter(y_power_pred)

In [ ]:
Counter(y_default_pred)

In [ ]:
# Train neural network classifier for optimizing accuracy vs power
val_nn = MLPClassifier(hidden_layer_sizes=(5,2), max_iter=10000)
val_nn.fit(X_tr, y_tr)
probs = val_nn.predict_proba(X_te)[:, 1]
acc = np.zeros(len(cutvals))
power = np.zeros(len(cutvals))
for j in range(len(cutvals)):
    k = cutvals[j]
    tmp = (probs > k).astype(int)
    acc[j] = np.sum(tmp == y_te) / len(y_te)
    tau0 = np.mean(tmp[y_te == 0] == 1) # P_{X ~ q}(h(X) = 1 | Y = 0)
    tau1 = np.mean(tmp[y_te == 1] == 0) # P_{X ~ p}(h(X) = 0 | Y = 1)
    if int(tau1) == 1:
        power[j] = 0
    else:
        power[j] = (1 - tau0 - tau1) / np.sqrt(tau0 * (1 - tau0) / n0 + tau1 * (1 - tau1) / n1)




# prediction
y_acc_pred = (test_probs > acc_cut_opt).astype(int)
y_power_pred = (test_probs > power_cut_opt).astype(int)
y_default_pred = (test_probs > 0.5).astype(int)

tau0_a = np.mean(y_acc_pred[y_test == 0] == 1) # P_{X ~ q}(h(X) = 1)
tau1_a = np.mean(y_acc_pred[y_test == 1] == 0) # P_{X ~ q}(h(X) = 0)
stat_a[i] = (1 - tau0_a - tau1_a) / np.sqrt((tau0_a*(1-tau0_a)/test_n0) + (tau1_a*(1-tau1_a)/test_n1))

tau0_d = np.mean(y_default_pred[y_test == 0] == 1) # P_{X ~ q}(h(X) = 1)
tau1_d = np.mean(y_default_pred[y_test == 1] == 0) # P_{X ~ q}(h(X) = 0)
stat_d[i] = (1 - tau0_d - tau1_d) / np.sqrt((tau0_d*(1-tau0_d)/test_n0) + (tau1_d*(1-tau1_d)/test_n1))

tau0_p = np.mean(y_power_pred[y_test == 0] == 1) # P_{X ~ q}(h(X) = 1)
tau1_p = np.mean(y_power_pred[y_test == 1] == 0) # P_{X ~ q}(h(X) = 0)
stat_p[i] = (1 - tau0_p - tau1_p) / np.sqrt((tau0_p*(1-tau0_p)/test_n0) + (tau1_p*(1-tau1_p)/test_n1))

print("="* 30)
print("acc cut opt: ", acc_cut_opt)
print("pwr cut opt: ", power_cut_opt)
print("Stat of acc: " , stat_a[i])
print("Stat of default: ", stat_d[i])
print("Stat of power: ", stat_p[i])